In [1]:
!pip install matplotlib torch_geometric torch gdown --quiet

In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 81 (delta 1), reused 1 (delta 1), pack-reused 77 (from 1)
Receiving objects: 100% (81/81), 105.83 MiB | 38.21 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
%cd hackaton/

/home/onyxia/work/DL-Hackathon/hackaton


In [4]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


Retrieving folder contents
Retrieving folder 1wcUVBNQkZ04zStXkglXSgERfIvjSHJiL A
Processing file 1C8sjkO6JS0j2SyVwQ07m8PhQ-pHpuI78 test.json.gz
Processing file 12N11n8gufNA_C1ns-1IeBseBHgrSfRI1 train.json.gz
Retrieving folder 1Tj5YoYYDDXjDxxi-cywZgoDkT0b1Qbz- B
Processing file 11GBlrXMdP3HSD60w-56Tu6rbGkR-Ifww test.json.gz
Processing file 13vp-Kwef3UgAwMG-dokGwKyARym9iqtL train.json.gz
Retrieving folder 1e3B_tBMd693Iwv8x3zRR9c47l5yt_5ey C
Processing file 18XVe65ZsQ0PDLCqQa4WmneVhyfjGcXmT test.json.gz
Processing file 1z5lvG2CytbLQZt7Jmo9BopzFd0pKejEj train.json.gz
Retrieving folder 1cvM0eZwpD4gzjo44_zdodxudVBMrLza1 D
Processing file 1Gna_dHnBLX8vKaYGAAqAbw5QPerrNK1u test.json.gz
Processing file 1Pc-6LMML80-AgEoLVs2Q5hLtmR_rTEek train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1C8sjkO6JS0j2SyVwQ07m8PhQ-pHpuI78
From (redirected): https://drive.google.com/uc?

In [5]:
!ls -lh datasets

total 16K
drwxr-sr-x 2 onyxia users 4.0K May 27 07:04 A
drwxr-sr-x 2 onyxia users 4.0K May 27 07:05 B
drwxr-sr-x 2 onyxia users 4.0K May 27 07:05 C
drwxr-sr-x 2 onyxia users 4.0K May 27 07:05 D


In [4]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [5]:
# Modification de GraphDataset pour récupérer les indices (ajout de get)

import gzip
import json
import torch
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader

class GraphDataset(Dataset):
    def __init__(self, filename, transform=None, pre_transform=None):
        self.raw = filename
        self.num_graphs, self.graphs_dicts = self._count_graphs() 
        super().__init__(None, transform, pre_transform)

    def len(self):
        return self.num_graphs  
    
    def get(self, idx):
        data = dictToGraphObject(self.graphs_dicts[idx])
        data.idx = idx  # 🔥 Ajoute l'indice global dans l'objet Data
        return data

    def _count_graphs(self):
        with gzip.open(self.raw, "rt", encoding="utf-8") as f:
            graphs_dicts = json.load(f)
            return len(graphs_dicts), graphs_dicts

def dictToGraphObject(graph_dict):
    edge_index = torch.tensor(graph_dict["edge_index"], dtype=torch.long)
    edge_attr = torch.tensor(graph_dict["edge_attr"], dtype=torch.float) if graph_dict["edge_attr"] else None
    num_nodes = graph_dict["num_nodes"]
    y = torch.tensor(graph_dict["y"][0], dtype=torch.long) if graph_dict["y"] is not None else None
    return Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=num_nodes, y=y)

In [6]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [7]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [8]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [9]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [10]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [11]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (GCOD)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


In [20]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/C/train.json.gz
Path to the test dataset [None]:  datasets/C/test.json.gz
Number of checkpoints to save during training [None]:  50
Which GPU to use if any [1]:  
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn
Dropout ratio [0.0]:  
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [10]:  50
Baseline mode: 1 (CE), 2 (GCOD) [1]:  2
Noise probability p (used if baseline_mode=2) [0.2]:  


Arguments received:
train_path: datasets/C/train.json.gz
test_path: datasets/C/test.json.gz
num_checkpoints: 50
device: 1
gnn: gcn
drop_ratio: 0.0
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 50
baseline_mode: 2
noise_prob: 0.2


In [21]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    # On initialise tout (criterion et optimizer) dans la boucle après la définition de full_dataset, car on a besoin du nombre de sample
    pass
else:
    criterion = torch.nn.CrossEntropyLoss()

In [22]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [23]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [24]:
class IndexedSubDataset(Dataset):
    def __init__(self, subset):
        self.subset = subset

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, i):
        data = self.subset[i]
        data.idx = i  # 👈 réindexe localement
        return data

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCODLoss(nn.Module):
    #def __init__(self, num_samples, num_classes=6, alpha=1.0, beta=1.0):
    def __init__(self, num_classes=6, alpha=1.0, beta=1.0):
        super().__init__()
        self.num_classes = num_classes
        self.alpha = alpha
        self.beta = beta

        # u est un vecteur trainable de taille num_samples
        #self.register_parameter("u", nn.Parameter(torch.ones(num_samples)))

    def forward(self, logits, u_batch, y_onehot, y_soft, a_train):
        """
        logits: (B, C)
        u_batch: (B,)
        y_onehot: (B, C)
        y_soft: (B, C)
        a_train: float
        """
        B, C = logits.shape

        u_diag_y = u_batch.unsqueeze(1) * y_onehot  # (B, C)

        # L1: weighted cross-entropy with soft labels
        logits_mod = logits + a_train * u_diag_y
        L1 = F.cross_entropy(logits_mod, y_soft.argmax(dim=1))

        # L2: squared norm alignment
        y_pred_onehot = torch.zeros_like(y_onehot)
        y_pred_onehot.scatter_(1, logits.argmax(dim=1, keepdim=True), 1)
        L2 = (1 / C) * ((y_pred_onehot + u_diag_y - y_onehot).pow(2).sum(dim=1).mean())

        # L3: KL regularization
        diag_logits_y = (logits * y_onehot).sum(dim=1)
        L = torch.log(torch.sigmoid(diag_logits_y) + 1e-12)
        sigma_neg_log_u = 1.0 / (1.0 + u_batch)

        P = torch.sigmoid(L)
        Q = sigma_neg_log_u.clamp(min=1e-6, max=1 - 1e-6)
        L3 = (1 - a_train) * (P * (torch.log(P + 1e-12) - torch.log(Q))).mean()

        return L1 + L3, L2

In [26]:
def train(train_loader, model, optimizer_theta, optimizer_u, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        batch = batch.to(device)

        optimizer_theta.zero_grad()
        optimizer_u.zero_grad()

        logits = model(batch)


        # Définition de la matrice de transition T
        C = 6  # nombre de classes
        eta = 0.2  # probabilité de bruit
        T = torch.eye(C, device=device) * (1 - eta) + (1 - torch.eye(C, device=device)) * (eta / (C - 1))

        y = batch.y
        # Supposons que tu as la matrice de bruit T déjà définie, sur le device
        # labels_bruites = y (tes labels bruités)
        y_soft = T[y]  # T est ta matrice de bruit (C x C), y shape (B,), y_soft shape (B, C)

        # Récupère les indices des graphes dans le batch
        #graph_indices = batch.idx.to(criterion.u.device)        
        #u_batch = criterion.u[graph_indices]

        # Récupération des indices (par exemple avec batch.ptr)
        graph_indices = batch.idx.to(device)
        u_batch = u[graph_indices]  # 🔁 u est passé de l'extérieur

        u_batch.data.clamp_(0, 1)
        
        # ➕ Calcul des prédictions pour accuracy
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
        train_acc = correct / total

        # ➕ Calcul des pertes
        loss_theta, loss_u = criterion(logits, u_batch, y_onehot, y_soft, train_acc)

        # ➕ Backpropagation
        loss_theta.backward(retain_graph=True)
        optimizer_theta.step()

        loss_u.backward()
        optimizer_u.step()

        # ✅ Clamp u entre 0 et 1 pour éviter qu’il devienne négatif ou explose
        u.data.clamp_(0, 1)


        total_loss += loss_theta.item()

    avg_loss = total_loss / len(train_loader)
    final_train_acc = correct / total
    return avg_loss, final_train_acc


In [ ]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_dataset = IndexedSubDataset(train_dataset)

    if args.baseline_mode == 2:

        u = nn.Parameter(torch.zeros(len(train_dataset), device=device))
        optimizer_u = torch.optim.Adam([u], lr=1)
        optimizer_theta = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
        
        criterion = GCODLoss(num_classes=6, alpha=1.0, beta=1.0)


    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)  # shuffle=False obligatoire
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer_theta, optimizer_u, criterion, device
        )

        val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating eval graphs: 100%|██████████| 48/48 [00:08<00:00,  5.34batch/s]
Epoch 1/50, Loss: 1.1253, Train Acc: 0.5158, Val Acc: 0.4622
Epoch 1/50, Loss: 1.1253, Train Acc: 0.5158, Val Acc: 0.4622


Epoch 1/50, Loss: 1.1253, Train Acc: 0.5158, Val Acc: 0.4622
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_C_best.pth


Iterating eval graphs: 100%|██████████| 48/48 [00:08<00:00,  5.46batch/s]
Epoch 2/50, Loss: 0.6714, Train Acc: 0.6353, Val Acc: 0.2871
Epoch 2/50, Loss: 0.6714, Train Acc: 0.6353, Val Acc: 0.2871


Epoch 2/50, Loss: 0.6714, Train Acc: 0.6353, Val Acc: 0.2871


Iterating eval graphs: 100%|██████████| 48/48 [00:09<00:00,  5.28batch/s]
Epoch 3/50, Loss: 0.6216, Train Acc: 0.6737, Val Acc: 0.5944
Epoch 3/50, Loss: 0.6216, Train Acc: 0.6737, Val Acc: 0.5944


...

Epoch 27/50, Loss: 0.0751, Train Acc: 0.9613, Val Acc: 0.7480
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_C_best.pth


Iterating eval graphs: 100%|██████████| 48/48 [00:32<00:00,  1.46batch/s]
Epoch 28/50, Loss: 0.0588, Train Acc: 0.9728, Val Acc: 0.7415
Epoch 28/50, Loss: 0.0588, Train Acc: 0.9728, Val Acc: 0.7415


Epoch 28/50, Loss: 0.0588, Train Acc: 0.9728, Val Acc: 0.7415


Iterating eval graphs: 100%|██████████| 48/48 [00:09<00:00,  5.23batch/s]
Epoch 29/50, Loss: 0.0623, Train Acc: 0.9723, Val Acc: 0.7428
Epoch 29/50, Loss: 0.0623, Train Acc: 0.9723, Val Acc: 0.7428


Epoch 29/50, Loss: 0.0623, Train Acc: 0.9723, Val Acc: 0.7428


In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

In [ ]:
import tarfile
import os

def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.

    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")

# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/submission"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/submission.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)